## N-gram on registry strings

### 1. Used to save the registry_features_list for both normal and ransom

In [2]:
import pickle
import itertools
import sys


ransome_features = './feature_ransom.txt'
normal_features = './feature_normal.txt'

In [3]:
def features_fields(features_txt):
    features_list = []
    stop =sum(1 for line in open(features_txt, 'r'))
    # feature = open("features_normal.txt","r",encoding="utf-8")
    with open(features_txt,"r") as fr:
        feature = fr.readlines()    
        for ind in itertools.islice(feature , stop):
            fields = ind.split(':', 1)
            features_list.append(fields[1])
        return features_list

In [4]:
normal_features_list = features_fields('../dataset/features_normal.txt')

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/features_normal.txt'

In [4]:
ransom_features_list = features_fields('../dataset/features_ransom.txt')

In [5]:
# extract the regkey parts from both features_list
for feature in normal_features_list:
    if feature[:7] != 'regkey':
        normal_features_list.remove(feature)
for feature in ransom_features_list:
    if feature[:7] != 'regkey':
        ransom_features_list.remove(feature)


In [6]:
# save the registry features with titles
with open('../dataset/registry_features/nor_registry_features.pkl','wb') as nw:
    pickle.dump(normal_features_list, nw)
with open('../dataset/registry_features/ran_registry_features.pkl','wb') as rw:
    pickle.dump(ransom_features_list, rw)

### 2. Used to analyse the saved features results

In [7]:
from nltk import ngrams
from nltk import FreqDist
import pandas as pd

def ngram_fre_analysis(n, features_list):
    ngrams_list = []
    for feature in features_list:
        grams = ngrams(feature.split("\\"),n)
        for gram in grams:
            ngrams_list.append(gram)
    
    ngrams_freqdist = FreqDist(ngrams_list)
    return ngrams_freqdist

In [8]:
# read the normal_features_list
with open('../dataset/registry_features/nor_registry_features.pkl','rb') as wr:
    normal_features_list = pickle.load(wr)

# read the ransom_featuers_list
with open('../dataset/registry_features/ran_registry_features.pkl','rb') as wr:
    ransom_features_list = pickle.load(wr)

In [9]:
# get the fre of three ngrams for two sets of features
thr_ngrams_freqdist_nor = ngram_fre_analysis(3, normal_features_list)
thr_ngrams_freqdist_ran = ngram_fre_analysis(3, ransom_features_list)

In [10]:
dict(thr_ngrams_freqdist_nor)

{('regkey_opened_HKEY_LOCAL_MACHINE', 'SOFTWARE', 'Wow6432Node'): 143,
 ('SOFTWARE', 'Wow6432Node', 'Microsoft'): 1237,
 ('Wow6432Node', 'Microsoft', 'Windows'): 1173,
 ('Microsoft', 'Windows', 'CurrentVersion'): 1368,
 ('Windows', 'CurrentVersion', 'explorer'): 1163,
 ('CurrentVersion', 'explorer', 'FolderDescriptions'): 1151,
 ('explorer',
  'FolderDescriptions',
  '{724EF170-A42D-4FEF-9F26-B60E846FBA4F}'): 8,
 ('FolderDescriptions',
  '{724EF170-A42D-4FEF-9F26-B60E846FBA4F}',
  'PropertyBag\n'): 1,
 ('Wow6432Node', 'Microsoft', 'Cryptography'): 47,
 ('Microsoft', 'Cryptography', 'OID'): 42,
 ('Cryptography', 'OID', 'EncodingType 0'): 35,
 ('OID', 'EncodingType 0', 'CryptSIPDllVerifyIndirectData'): 7,
 ('EncodingType 0',
  'CryptSIPDllVerifyIndirectData',
  '{DE351A42-8E59-11D0-8C47-00C04FC295EE}\n'): 1,
 ('regkey_opened_HKEY_LOCAL_MACHINE', 'Software', 'Microsoft'): 40,
 ('Software', 'Microsoft', 'Cryptography'): 8,
 ('Microsoft', 'Cryptography', 'Providers'): 11,
 ('Cryptography', 

### 3. Visualized Analysis for Three ngrams data

In [11]:
fd_nor = pd.DataFrame(thr_ngrams_freqdist_nor.keys())
fd_nor['frequency'] = thr_ngrams_freqdist_nor.values()
cols = [0,1,2]
fd_nor['features'] = fd_nor[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
fd_nor = fd_nor.sort_values(by=['frequency'], ascending=False)
fd_nor = fd_nor[['features','frequency']]
fd_nor.reset_index()
fd_nor

,features,frequency
2957,"Program Files,XMind ZEN,resources",2310
2958,"XMind ZEN,resources,app.asar.unpacked",2308
2959,"resources,app.asar.unpacked,node_modules",2300
2956,"file_created_C:,Program Files,XMind ZEN",1926
3,"Microsoft,Windows,CurrentVersion",1368
...,...,...
2312,"Nls,Language Groups,b\n",1
2311,"FolderDescriptions,{D0384E7D-BAC3-4797-8F14-CB...",1
2310,"SystemFileAssociations,system,DocObject\n",1
2309,"regkey_read_HKEY_CURRENT_USER,manifest_auto_fi...",1


In [12]:
fd_ran = pd.DataFrame(thr_ngrams_freqdist_ran.keys())
fd_ran['frequency'] = thr_ngrams_freqdist_ran.values()
cols = [0,1,2]
fd_ran['features'] = fd_ran[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
fd_ran = fd_ran.sort_values(by=['frequency'], ascending=False)
fd_ran.reset_index()
fd_ran = fd_ran[['features','frequency']]
fd_ran

,features,frequency
10,"Microsoft,Windows,CurrentVersion",1639
8,"SOFTWARE,Wow6432Node,Microsoft",1504
1425,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1259
9,"Wow6432Node,Microsoft,Windows",1247
4226,"Users,newt,AppData",1225
...,...,...
3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1
3005,"runas,command,command\n",1
3004,"Applications,NTVDM.exe,NoStaticDefaultVerb\n",1
3003,"GACChangeNotification,Default,Microsoft.Visual...",1


add rank for the ransomware behavior dataframe 

In [13]:
ransom_rank = range(1,len(fd_ran)+1)
fd_ran['rank'] = ransom_rank
fd_ran

,features,frequency,rank
10,"Microsoft,Windows,CurrentVersion",1639,1
8,"SOFTWARE,Wow6432Node,Microsoft",1504,2
1425,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1259,3
9,"Wow6432Node,Microsoft,Windows",1247,4
4226,"Users,newt,AppData",1225,5
...,...,...,...
3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1,7370
3005,"runas,command,command\n",1,7371
3004,"Applications,NTVDM.exe,NoStaticDefaultVerb\n",1,7372
3003,"GACChangeNotification,Default,Microsoft.Visual...",1,7373


add rank for the normal behavior dataframe

In [14]:
normal_rank = range(1,len(fd_nor)+1)
fd_nor['rank'] = normal_rank
fd_nor

,features,frequency,rank
2957,"Program Files,XMind ZEN,resources",2310,1
2958,"XMind ZEN,resources,app.asar.unpacked",2308,2
2959,"resources,app.asar.unpacked,node_modules",2300,3
2956,"file_created_C:,Program Files,XMind ZEN",1926,4
3,"Microsoft,Windows,CurrentVersion",1368,5
...,...,...,...
2312,"Nls,Language Groups,b\n",1,6408
2311,"FolderDescriptions,{D0384E7D-BAC3-4797-8F14-CB...",1,6409
2310,"SystemFileAssociations,system,DocObject\n",1,6410
2309,"regkey_read_HKEY_CURRENT_USER,manifest_auto_fi...",1,6411


find the unique features in ransome behavior while not in normal behavior

In [15]:
fd_ran[~fd_ran.features.isin(fd_nor['features'])].reset_index()

,index,features,frequency,rank
0,4231,"Python27,tcl,tcl8.5",366,11
1,4232,"tcl,tcl8.5,msgs",342,13
2,4242,"Users,All Users,Microsoft",275,17
3,6465,"file_deleted_C:,Python27,tcl",220,21
4,6458,"file_deleted_C:,Python27,Lib",202,24
...,...,...,...,...
5752,3007,"FolderDescriptions,{2B0F765D-C0E9-4171-908E-08...",1,7369
5753,3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1,7370
5754,3005,"runas,command,command\n",1,7371
5755,3003,"GACChangeNotification,Default,Microsoft.Visual...",1,7373


compare the ranks for the some features in two datasets

In [16]:
thr_feature_ranks = fd_ran[['features','rank']].merge(fd_nor[['features','rank']], on=['features'])
thr_feature_ranks.iloc[:194,:]


,features,rank_x,rank_y
0,"Microsoft,Windows,CurrentVersion",1,5
1,"SOFTWARE,Wow6432Node,Microsoft",2,6
2,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",3,10
3,"Wow6432Node,Microsoft,Windows",4,7
4,"Users,newt,AppData",5,26
...,...,...,...
189,"explorer,FolderDescriptions,{1777F761-68AD-4D8...",330,212
190,"explorer,FolderDescriptions,{A75D362E-50FC-4FB...",331,181
191,"EncodingType 0,CertDllCreateCertificateChainEn...",332,331
192,"Wow6432Node,CLSID,{1F3427C8-5C10-4210-AA03-2EE...",333,391


compare the frequencies for the some features in two datasets

In [17]:
thr_feature_freqs = fd_ran[['features','frequency']].merge(fd_nor[['features','frequency']], on=['features'])
thr_feature_freqs


,features,frequency_x,frequency_y
0,"Microsoft,Windows,CurrentVersion",1639,1368
1,"SOFTWARE,Wow6432Node,Microsoft",1504,1237
2,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1259,1091
3,"Wow6432Node,Microsoft,Windows",1247,1173
4,"Users,newt,AppData",1225,161
...,...,...,...
1612,"FolderDescriptions,{4BD8D571-6D19-48D3-BE97-42...",1,1
1613,"Lsa,FipsAlgorithmPolicy,Enabled\n",1,1
1614,"Control,Lsa,FipsAlgorithmPolicy",1,1
1615,"FolderDescriptions,{054FAE61-4DD8-4787-80B6-09...",1,1


In [18]:
thr_feature_freqs[thr_feature_freqs['frequency_x']>191]

,features,frequency_x,frequency_y
0,"Microsoft,Windows,CurrentVersion",1639,1368
1,"SOFTWARE,Wow6432Node,Microsoft",1504,1237
2,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1259,1091
3,"Wow6432Node,Microsoft,Windows",1247,1173
4,"Users,newt,AppData",1225,161
5,"Windows,CurrentVersion,explorer",1190,1163
6,"CurrentVersion,explorer,FolderDescriptions",1174,1151
7,"newt,AppData,Local",929,121
8,"file_created_C:,Users,newt",490,31
9,"AppData,Local,Temp",418,108


### 4. Visualized Analysis for Four ngrams data

In [19]:
# get the fre of four ngrams for two sets of features
four_ngrams_freqdist_nor = ngram_fre_analysis(4, normal_features_list)
four_ngrams_freqdist_ran = ngram_fre_analysis(4, ransom_features_list)

In [20]:
four_ngrams_freqdist_nor

FreqDist({('Program Files', 'XMind ZEN', 'resources', 'app.asar.unpacked'): 2308, ('XMind ZEN', 'resources', 'app.asar.unpacked', 'node_modules'): 2300, ('file_created_C:', 'Program Files', 'XMind ZEN', 'resources'): 1888, ('SOFTWARE', 'Wow6432Node', 'Microsoft', 'Windows'): 1173, ('Wow6432Node', 'Microsoft', 'Windows', 'CurrentVersion'): 1173, ('Microsoft', 'Windows', 'CurrentVersion', 'explorer'): 1163, ('Windows', 'CurrentVersion', 'explorer', 'FolderDescriptions'): 1151, ('regkey_read_HKEY_LOCAL_MACHINE', 'SOFTWARE', 'Wow6432Node', 'Microsoft'): 1090, ('resources', 'app.asar.unpacked', 'node_modules', 'core-js'): 784, ('app.asar.unpacked', 'node_modules', 'core-js', 'library'): 378, ...})

In [21]:
four_ngrams_freqdist_ran

FreqDist({('regkey_read_HKEY_LOCAL_MACHINE', 'SOFTWARE', 'Wow6432Node', 'Microsoft'): 1255, ('SOFTWARE', 'Wow6432Node', 'Microsoft', 'Windows'): 1247, ('Wow6432Node', 'Microsoft', 'Windows', 'CurrentVersion'): 1244, ('Microsoft', 'Windows', 'CurrentVersion', 'explorer'): 1190, ('Windows', 'CurrentVersion', 'explorer', 'FolderDescriptions'): 1174, ('Users', 'newt', 'AppData', 'Local'): 929, ('file_created_C:', 'Users', 'newt', 'AppData'): 474, ('newt', 'AppData', 'Local', 'Temp'): 418, ('Software', 'Microsoft', 'Windows', 'CurrentVersion'): 344, ('Python27', 'tcl', 'tcl8.5', 'msgs'): 342, ...})

### 5. Visualized analysis for four ngrams data

In [22]:
fd_nor_four = pd.DataFrame(four_ngrams_freqdist_nor.keys())
fd_nor_four['frequency'] = four_ngrams_freqdist_nor.values()
cols = [0,1,2,3]
fd_nor_four['features'] = fd_nor_four[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
fd_nor_four = fd_nor_four.sort_values(by=['frequency'], ascending=False)
fd_nor_four = fd_nor_four[['features','frequency']]
fd_nor_four.reset_index()
fd_nor_four

,features,frequency
2890,"Program Files,XMind ZEN,resources,app.asar.unp...",2308
2891,"XMind ZEN,resources,app.asar.unpacked,node_mod...",2300
2889,"file_created_C:,Program Files,XMind ZEN,resources",1888
2,"Wow6432Node,Microsoft,Windows,CurrentVersion",1173
1,"SOFTWARE,Wow6432Node,Microsoft,Windows",1173
...,...,...
2234,"explorer,FolderDescriptions,{DE974D24-D9C6-4D3...",1
2233,"@C:,Windows,system32,recdisc.exe,-2000\n",1
2232,"CLSID,{DFFACDC5-679F-4156-8947-C5C76BC0B67F},S...",1
2231,"explorer,FolderDescriptions,{B6EBFB86-6907-413...",1


create the feature dataframe for ransom behavior

In [23]:
fd_ran_four = pd.DataFrame(four_ngrams_freqdist_ran.keys())
fd_ran_four['frequency'] = four_ngrams_freqdist_ran.values()
cols = [0,1,2,3]
fd_ran_four['features'] = fd_ran_four[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
fd_ran_four = fd_ran_four.sort_values(by=['frequency'], ascending=False)
fd_ran_four.reset_index()
fd_ran_four = fd_ran_four[['features','frequency']]
fd_ran_four

,features,frequency
1451,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1255
7,"SOFTWARE,Wow6432Node,Microsoft,Windows",1247
8,"Wow6432Node,Microsoft,Windows,CurrentVersion",1244
15,"Microsoft,Windows,CurrentVersion,explorer",1190
16,"Windows,CurrentVersion,explorer,FolderDescript...",1174
...,...,...
2958,"explorer,FolderDescriptions,{054FAE61-4DD8-478...",1
2957,"explorer,FolderDescriptions,{9E3995AB-1F9C-4F1...",1
2956,"explorer,FolderDescriptions,{B88F4DAA-E7BD-49A...",1
2955,"explorer,FolderDescriptions,{D9DC8A3B-B784-432...",1


create the frequency dataframe for normal features

In [24]:
fd_nor_four = pd.DataFrame(four_ngrams_freqdist_nor.keys())
fd_nor_four['frequency'] = four_ngrams_freqdist_nor.values()
cols = [0,1,2,3]
fd_nor_four['features'] = fd_nor_four[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
fd_nor_four = fd_nor_four.sort_values(by=['frequency'], ascending=False)
fd_nor_four.reset_index()
fd_nor_four = fd_nor_four[['features','frequency']]
fd_nor_four

,features,frequency
2890,"Program Files,XMind ZEN,resources,app.asar.unp...",2308
2891,"XMind ZEN,resources,app.asar.unpacked,node_mod...",2300
2889,"file_created_C:,Program Files,XMind ZEN,resources",1888
2,"Wow6432Node,Microsoft,Windows,CurrentVersion",1173
1,"SOFTWARE,Wow6432Node,Microsoft,Windows",1173
...,...,...
2234,"explorer,FolderDescriptions,{DE974D24-D9C6-4D3...",1
2233,"@C:,Windows,system32,recdisc.exe,-2000\n",1
2232,"CLSID,{DFFACDC5-679F-4156-8947-C5C76BC0B67F},S...",1
2231,"explorer,FolderDescriptions,{B6EBFB86-6907-413...",1


create the rank for two dataframe

In [25]:
normal_rank = range(1,len(fd_nor_four)+1)
fd_nor_four['rank'] = normal_rank
fd_nor_four

ransom_rank = range(1,len(fd_ran_four)+1)
fd_ran_four['rank'] = ransom_rank
fd_ran_four


,features,frequency,rank
1451,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1255,1
7,"SOFTWARE,Wow6432Node,Microsoft,Windows",1247,2
8,"Wow6432Node,Microsoft,Windows,CurrentVersion",1244,3
15,"Microsoft,Windows,CurrentVersion,explorer",1190,4
16,"Windows,CurrentVersion,explorer,FolderDescript...",1174,5
...,...,...,...
2958,"explorer,FolderDescriptions,{054FAE61-4DD8-478...",1,7312
2957,"explorer,FolderDescriptions,{9E3995AB-1F9C-4F1...",1,7313
2956,"explorer,FolderDescriptions,{B88F4DAA-E7BD-49A...",1,7314
2955,"explorer,FolderDescriptions,{D9DC8A3B-B784-432...",1,7315


In [26]:
four_feature_ranks = fd_ran_four[['features','rank']].merge(fd_nor_four[['features','rank']], on=['features'])
four_feature_ranks.iloc[:194,:]

,features,rank_x,rank_y
0,"regkey_read_HKEY_LOCAL_MACHINE,SOFTWARE,Wow643...",1,8
1,"SOFTWARE,Wow6432Node,Microsoft,Windows",2,5
2,"Wow6432Node,Microsoft,Windows,CurrentVersion",3,4
3,"Microsoft,Windows,CurrentVersion,explorer",4,6
4,"Windows,CurrentVersion,explorer,FolderDescript...",5,7
...,...,...,...
189,"OID,EncodingType 0,CertDllCreateCertificateCha...",334,326
190,"CurrentVersion,explorer,FolderDescriptions,{3E...",335,124
191,"CurrentVersion,explorer,FolderDescriptions,{17...",337,189
192,"CurrentVersion,explorer,FolderDescriptions,{7B...",339,272


### Find the unique strings in ransomware activities

#### Four-Gram analysis

In [27]:
unique_fd_ran_four = fd_ran_four[~fd_ran_four.features.isin(fd_nor_four['features'])].reset_index()
unique_fd_ran_four

,index,features,frequency,rank
0,4293,"Python27,tcl,tcl8.5,msgs",342,10
1,4325,"Local,Microsoft,Windows,Temporary Internet Files",168,19
2,4326,"Microsoft,Windows,Temporary Internet Files,Con...",167,20
3,6518,"file_deleted_C:,Python27,tcl,tcl8.5",150,22
4,29,"SOFTWARE,MICROSOFT,CTF,TIP",147,23
...,...,...,...,...
5758,2964,"Internet Settings,Zones,1,CurrentLevel\n",1,7307
5759,2963,"Trust,Initialization,{573E31F8-AABA-11D0-8CCB-...",1,7308
5760,2962,"Providers,Trust,Initialization,{573E31F8-AABA-...",1,7309
5761,2959,"explorer,FolderDescriptions,{054FAE61-4DD8-478...",1,7311


In [28]:
# save the result for three-ngram
unique_fd_ran_four.to_pickle('../dataset/N-Gram/unique_fd_four.pkl')

In [29]:
fd_ran_four[~fd_ran_four.features.isin(fd_nor_four['features'])][fd_ran_four['frequency']==1].reset_index()

<ipython-input-29-2df9c441eef4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fd_ran_four[~fd_ran_four.features.isin(fd_nor_four['features'])][fd_ran_four['frequency']==1].reset_index()


,index,features,frequency,rank
0,7018,"email,guest,data,msg_17.txt.WNCRYT\n",1,2055
1,7010,"tcl,tk8.5,msgs,ru.msg.WNCRYT\n",1,2056
2,7009,"Python27,Lib,guest,exception_hierarchy.txt\n",1,2057
3,4882,"file_created_C:,Python27,Tools,Scripts",1,2058
4,4890,"Python27,tcl,tcl8.5,http1.0",1,2059
...,...,...,...,...
4171,2964,"Internet Settings,Zones,1,CurrentLevel\n",1,7307
4172,2963,"Trust,Initialization,{573E31F8-AABA-11D0-8CCB-...",1,7308
4173,2962,"Providers,Trust,Initialization,{573E31F8-AABA-...",1,7309
4174,2959,"explorer,FolderDescriptions,{054FAE61-4DD8-478...",1,7311


In [30]:
fd_ran_four[~fd_ran_four.features.isin(fd_nor_four['features'])][fd_ran_four['frequency']<=191].reset_index()

<ipython-input-30-9f08b1495f5d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fd_ran_four[~fd_ran_four.features.isin(fd_nor_four['features'])][fd_ran_four['frequency']<=191].reset_index()


,index,features,frequency,rank
0,4325,"Local,Microsoft,Windows,Temporary Internet Files",168,19
1,4326,"Microsoft,Windows,Temporary Internet Files,Con...",167,20
2,6518,"file_deleted_C:,Python27,tcl,tcl8.5",150,22
3,29,"SOFTWARE,MICROSOFT,CTF,TIP",147,23
4,6504,"file_deleted_C:,Users,All Users,Microsoft",122,25
...,...,...,...,...
5757,2964,"Internet Settings,Zones,1,CurrentLevel\n",1,7307
5758,2963,"Trust,Initialization,{573E31F8-AABA-11D0-8CCB-...",1,7308
5759,2962,"Providers,Trust,Initialization,{573E31F8-AABA-...",1,7309
5760,2959,"explorer,FolderDescriptions,{054FAE61-4DD8-478...",1,7311


#### Three-Gram analysis

In [31]:
unique_fd_ran_three = fd_ran[~fd_ran.features.isin(fd_nor['features'])].reset_index()
unique_fd_ran_three

,index,features,frequency,rank
0,4231,"Python27,tcl,tcl8.5",366,11
1,4232,"tcl,tcl8.5,msgs",342,13
2,4242,"Users,All Users,Microsoft",275,17
3,6465,"file_deleted_C:,Python27,tcl",220,21
4,6458,"file_deleted_C:,Python27,Lib",202,24
...,...,...,...,...
5752,3007,"FolderDescriptions,{2B0F765D-C0E9-4171-908E-08...",1,7369
5753,3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1,7370
5754,3005,"runas,command,command\n",1,7371
5755,3003,"GACChangeNotification,Default,Microsoft.Visual...",1,7373


In [32]:
# save the result for three-ngram
unique_fd_ran_three.to_pickle('../dataset/N-Gram/unique_fd_three.pkl')

In [33]:
fd_ran[~fd_ran.features.isin(fd_nor['features'])][fd_ran['frequency']<=191].reset_index()

<ipython-input-33-7f1cb17efd37>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fd_ran[~fd_ran.features.isin(fd_nor['features'])][fd_ran['frequency']<=191].reset_index()


,index,features,frequency,rank
0,4272,"Python27,Lib,guest",175,27
1,4266,"Microsoft,Windows,Temporary Internet Files",168,28
2,4267,"Windows,Temporary Internet Files,Content.IE5",167,29
3,32,"SOFTWARE,MICROSOFT,CTF",147,33
4,33,"MICROSOFT,CTF,TIP",147,34
...,...,...,...,...
5747,3007,"FolderDescriptions,{2B0F765D-C0E9-4171-908E-08...",1,7369
5748,3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1,7370
5749,3005,"runas,command,command\n",1,7371
5750,3003,"GACChangeNotification,Default,Microsoft.Visual...",1,7373


In [34]:
fd_ran[~fd_ran.features.isin(fd_nor['features'])][fd_ran['frequency']==1].reset_index()

<ipython-input-34-38f9251836ab>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fd_ran[~fd_ran.features.isin(fd_nor['features'])][fd_ran['frequency']==1].reset_index()


,index,features,frequency,rank
0,5040,"User Data,OriginTrials,~SDB3B4.tmp\n",1,2208
1,7208,"svlhbcnth,RMS^hlRFOUVbZLKUWW_YI>Sdeb,VV]s\n",1,2209
2,7207,"strings_}ttqkpz}}wkht}pcdehbgae[U^da[XQ,svlhbc...",1,2210
3,5038,"Content.IE5,LXGDWYKT,BBACPiN[1].png\n",1,2211
4,5035,"Content.IE5,3DL0D11W,BBLLeL3[1].jpg\n",1,2212
...,...,...,...,...
4045,3007,"FolderDescriptions,{2B0F765D-C0E9-4171-908E-08...",1,7369
4046,3006,"{CEBFF5CD-ACE2-4F4F-9178-9926F41749EA},Count,Z...",1,7370
4047,3005,"runas,command,command\n",1,7371
4048,3003,"GACChangeNotification,Default,Microsoft.Visual...",1,7373
